<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

Student: Oleg Navolotsky / Наволоцкий Олег  
Stepik: https://stepik.org/users/2403189  
Telegram: [@mehwhatever0](https://t.me/mehwhatever0)

Some used software versions:
- NumPy 1.18.5
- NLTK 3.5
- Gensim 3.8.3
- Python 3.8.3 (default, Jul  2 2020, 17:30:36) \[MSC v.1916 64 bit (AMD64)] :: Anaconda, Inc. on win32
- Windows 10 Pro 1909, build 18363.535

Hardware:
- i5 2500 8 gb

In [1]:
import os
import random

import numpy as np

SEED = 0

def enable_reproducibility(seed=SEED):
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [2]:
!wget -nc https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1 -O SO_vectors_200.bin

File 'SO_vectors_200.bin' already there; not retrieving.
C:\Users\user0\anaconda3\envs\custom\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
C:\Users\user0\anaconda3\envs\custom\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\Users\user0\anaconda3\envs\custom\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


In [3]:
from gensim.models.keyedvectors import KeyedVectors

wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

C:\Users\user0\anaconda3\envs\custom\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [4]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [5]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [6]:
for i, (word, _) in enumerate(wv_embeddings.similar_by_word('dog', topn=5)):
    if word == 'cat':
        print(f"Входит: {i} место.")
        break
else:
    print("Ответ: не входит.")

Ответ: не входит.


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [7]:
import numpy as np
import re

# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass

    def tokenize(self, text):
        return re.findall(r'\w+', text)

tokenizer = MyTokenizer().tokenize

In [8]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    assert embeddings.vectors.shape[-1] == dim
    res_vec = np.zeros_like(embeddings.vectors[0])
    num = 0
    for word in tokenizer(question):
        if word in embeddings:
            res_vec += embeddings[word]
            num += 1
    return res_vec / num if num > 0 else res_vec

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [9]:
print(f'Ответ: {question_to_vec("I love neural networks", wv_embeddings, tokenizer)[2].round(2):.2f}.')

Ответ: -1.29.


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Ответ: **1**.

$Hits@K$ не учитывает позицию, а значит её значение максимально, если для кажого из $N$ вопросов позиция его дубликата $\le K$. Минимальное значение **DCG@K** — ноль, достигается при рангах все дубликатов $\gt K$. Для данных $Hits@47$ и $DCG@1$ теоретически достижимое значение разности $Hits@47 - DCG@1$ равно 1. Модель должна быть такой, что ранги дубликатов для всех $N$ вопросов удовлетворяют $2 \le rank\_q_{i}^{'} \le 47$.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

Ответ: $\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}} \approx$

In [10]:
import math

print('\u2248', round(1 / math.log2(10), 1))

≈ 0.3


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [11]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    dup_ranks = np.array(dup_ranks)
    assert np.all(dup_ranks > 0)
    hits_value = (dup_ranks <= k).mean()
    return hits_value    

In [12]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dup_ranks = np.array(dup_ranks)
    assert np.all(dup_ranks > 0)
    dcg_value = np.mean((dup_ranks <= k) / np.log2(1 + dup_ranks))
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [13]:
import pandas as pd

In [14]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown"]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"]]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [cands.index(q) + 1 for q, cands in zip(copy_answers, candidates_ranking)]
print(f"{dup_ranks = }")

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

dup_ranks = [2]
Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [15]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [16]:
# !unzip stackoverflow_similar_questions.zip

Считайте данные.

In [17]:
def read_corpus(filename):
    with open(filename, encoding='utf-8') as input_file:
        return [line.strip().split('\t') for line in input_file]

Нам понадобиться только файл validation.

In [18]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [19]:
len(validation_data)

3760

Размер нескольких первых строк

In [20]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [22]:
def rank_candidates(question, candidates, embeddings, preprocessor, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    q_vec_resh = question_to_vec(question, embeddings, preprocessor, dim).reshape(1, -1)
    return sorted(
        enumerate(candidates),
        key=lambda i_c: 1 - cosine_similarity(
            q_vec_resh,
            question_to_vec(i_c[1], embeddings, preprocessor, dim).reshape(1, -1)))

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [23]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [24]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks, end='\n\n')

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [25]:
# # должно вывести
# results = [[(1, 'C# create cookie from string and send it'),
#             (0, 'Convert Google results object (pure js) to Python object'),
#             (2, 'How to use jQuery AJAX for an outside domain?')],
#            [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
#             (*, 'select2 not displaying search results'), #скрыт
#             (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Ответ: 102.

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [26]:
from itertools import islice

from tqdm.notebook import tqdm


MAX_VAL_EX = 1000


def make_validation(embeddings, tokenizer, validation_data, max_validation_examples=MAX_VAL_EX):
    ranking = []
    for i, line in tqdm(
            enumerate(islice(validation_data, max_validation_examples)),
            total=max_validation_examples):
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings, tokenizer)
        ranking.append([r[0] for r in ranks].index(0) + 1)
    return ranking


def get_metrics(ranking, *, K=(1, 5, 10, 100, 500, 1000), show=True):
    metrics = {}
    for k in tqdm(K):
        dcg = dcg_score(ranking, k)
        hits = hits_count(ranking, k)
        if show:
            print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg, k, hits))
        metrics[f"DCG@{k}"] = dcg
        metrics[f"Hits@{k}"] = hits
    return metrics


class UpdatableListValueDictionary(dict):
    def update(self, other, **kwargs):
        if hasattr(other, 'keys'):
            for key in other.keys():
                self[key] = other[key]
        else:
            for key, value in other:
                self[key] = value
        for key, value in kwargs.items():
            self[key] = value
    
    def __setitem__(self, key, value):
        if isinstance(value, list):
            super().__setitem__(key, value)
            return
        if key in self:
            self[key].append(value)
        else:
            super().__setitem__(key, [value])

In [27]:
metrics = UpdatableListValueDictionary()
so_embeddings = wv_embeddings

## Experiment #1:
default tokenizer & stackoverflow embeddings

In [28]:
soe_based_ranking = make_validation(so_embeddings, tokenizer, validation_data)

In [29]:
metrics['experiment'] = "#1: def tknz & stackoverflow embds"
metrics.update(get_metrics(soe_based_ranking))

DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.879
DCG@1000: 0.444 | Hits@1000: 1.000



### Эмбеддинги, обученные на корпусе похожих вопросов

In [30]:
train_data = read_corpus('./data/train.tsv')


Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [31]:
from gensim.models import Word2Vec


def train_embeddings(data, *, window, min_count, size=200):
    enable_reproducibility()
    return Word2Vec(
        data,  # data for model to train on
        size=size,  # embedding vector size
        window=window,
        min_count=min_count  # consider words that occured at least given times
    ).wv


def prepare_data(data, preprocessor):
    return map(preprocessor, (' '.join(doc) for doc in data))

## Experiment #2:
default tokenizer & our own embeddings (default window (5), default min_count (5))

Размер окна оставлен таким, какой он стоит по умолчанию в сигнатуре конструктора. Обычно в таких библиотеках если установлено некое значение аргумента по умолчанию, то оно уже достаточно оптимальное в большинстве случаев и не является тем, что нужно подбирать и оптимизировать в первую очередь.

In [32]:
%%time
ex_2_embeddings_trained = train_embeddings(prepare_data(train_data, tokenizer), window=5, min_count=5)

Wall time: 28.2 s


In [33]:
our_embeddings_based_ranking = make_validation(ex_2_embeddings_trained, tokenizer, validation_data)

In [34]:
metrics['experiment'] = "#2: def tknz & our own embds (win 5, min_cnt 5)"
metrics.update(get_metrics(our_embeddings_based_ranking))

DCG@   1: 0.198 | Hits@   1: 0.198
DCG@   5: 0.238 | Hits@   5: 0.274
DCG@  10: 0.250 | Hits@  10: 0.313
DCG@ 100: 0.290 | Hits@ 100: 0.516
DCG@ 500: 0.324 | Hits@ 500: 0.783
DCG@1000: 0.347 | Hits@1000: 1.000



## Experiment #3:
default tokenizer + lowercase & stackoverflow embeddings

In [35]:
def preprocess(data):
    return tokenizer(data.lower())

In [36]:
soe_based_ranking = make_validation(so_embeddings, preprocess, validation_data)

In [37]:
metrics['experiment'] = "#3: def tknz + lowercase & stackoverflow embds"
metrics.update(get_metrics(soe_based_ranking))

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.525 | Hits@  10: 0.651
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000



## Experiment #4:
default tokenizer + lowercase & our own embeddings (default window (5), default min_count (5))

In [38]:
%%time
ex_4_embeddings_trained = train_embeddings(prepare_data(train_data, preprocess), window=5, min_count=5)

Wall time: 24 s


In [39]:
our_embeddings_based_ranking = make_validation(ex_4_embeddings_trained, preprocess, validation_data)

In [40]:
metrics['experiment'] = "#4: def tknz + lowercase & our own embds (win 5, min_cnt 5)"
metrics.update(get_metrics(our_embeddings_based_ranking))

DCG@   1: 0.261 | Hits@   1: 0.261
DCG@   5: 0.315 | Hits@   5: 0.361
DCG@  10: 0.332 | Hits@  10: 0.414
DCG@ 100: 0.370 | Hits@ 100: 0.604
DCG@ 500: 0.401 | Hits@ 500: 0.844
DCG@1000: 0.417 | Hits@1000: 1.000



В нашей задаче регистр не так важен, как, например, в Named Entity Recognition, что отражается в увеличении значении метрик при использовании lowercase.

Увеличение качества на предобученных эмбеддингах связано с тем, что они содержат только слова в lowercase:


In [41]:
print(
    f"{'Python' in so_embeddings = }",
    f"{'python'.lower() in so_embeddings = }",
    sep='\n')

'Python' in so_embeddings = False
'python'.lower() in so_embeddings = True


In [42]:
from string import ascii_uppercase

print(f"{len(so_embeddings.vocab) = }")
print(f"{ascii_uppercase = }")
for word in so_embeddings.vocab:
    if any(ch in word for ch in ascii_uppercase):
        print("Example of an non-lowercase word in the stackoverflow embeddings vocab:", word)
        break
else:
    print("There are only lowercase words in the stackoverflow embeddings vocab")

len(so_embeddings.vocab) = 1787145
ascii_uppercase = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
There are only lowercase words in the stackoverflow embeddings vocab


In [43]:
from itertools import islice

how_many_words_to_show = 50

soe_vocab_examples = list(islice(so_embeddings.vocab.keys(), 0, len(so_embeddings.vocab), int(len(so_embeddings.vocab) / how_many_words_to_show)))
print("stackoverflow embeddings vocab examples:", soe_vocab_examples, sep='\n')

stackoverflow embeddings vocab examples:
['use', 'diplayed', 'udpated', 'phpcr', '75x', 'unviewable', 'com/it-it/library/system', 'not-recommended', 'another_user', 'regusers', 'com/components/#dropdowns', 'rmiinterface', 'updatemask', '50+50', 'onfileopen', 'tophone', '9193', 'appusermodelids', 'ÑÐºÐ', '_levels', 'programmer/user', 'greenrectangle', 'intar', 'casescontroller', 'com/docs/security/', 'trasnaction', '/app-root/runtime/repo', 'nav-direction', 'jaredc', 'start+duration', 'com/en-us/sql/t-sql/language-elements/case-transact-sql', 'e103', 'anyway/', 'staa', 'limitedness', 'draw_size', 'com/border-image', 'art_sifra', 'xmlaolap4jutil', 'uibuttion', 'domagoj', 'check/apply', '10994058', 'make-account', 'yourmode', 'type/record', '823x776', 'somethod', 'num_order', '9/20/2014', 'filter14']



Увеличества качества с нашими эмбеддингами вероятно связано с тем, что при обучении они чаще встречают слова с одинаковым смыслом, которые в разном регистре воспринимались бы отдельными сущностями, что должно сильно влиять на их обучение, т.к. корпус мал, особенно в сравнении с целым stackoverflow. 

## Experiment #5:
word_tokenize from nltk + lowercase & stackoverflow embeddings

In [44]:
import nltk

nltk.download('punkt')

from nltk.tokenize import word_tokenize

def preprocess(data):
    return word_tokenize(data.lower())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [45]:
soe_based_ranking = make_validation(so_embeddings, preprocess, validation_data)

In [46]:
metrics['experiment'] = "#5: word_tokenize from nltk + lowercase & stackoverflow embds"
metrics.update(get_metrics(soe_based_ranking))

DCG@   1: 0.399 | Hits@   1: 0.399
DCG@   5: 0.487 | Hits@   5: 0.566
DCG@  10: 0.508 | Hits@  10: 0.633
DCG@ 100: 0.554 | Hits@ 100: 0.858
DCG@ 500: 0.569 | Hits@ 500: 0.969
DCG@1000: 0.573 | Hits@1000: 1.000



## Experiment #6:
word_tokenize from nltk + lowercase & our own embeddings (default window (5), default min_count (5))

In [47]:
%%time
ex_6_embeddings_trained = train_embeddings(prepare_data(train_data, preprocess), window=5, min_count=5)

Wall time: 4min 42s


In [48]:
our_embeddings_based_ranking = make_validation(ex_6_embeddings_trained, preprocess, validation_data)

In [49]:
metrics['experiment'] = "#6: word_tokenize from nltk + lowercase & our own embds (win 5, min_cnt 5)"
metrics.update(get_metrics(our_embeddings_based_ranking))

DCG@   1: 0.229 | Hits@   1: 0.229
DCG@   5: 0.282 | Hits@   5: 0.330
DCG@  10: 0.298 | Hits@  10: 0.378
DCG@ 100: 0.336 | Hits@ 100: 0.571
DCG@ 500: 0.370 | Hits@ 500: 0.838
DCG@1000: 0.387 | Hits@1000: 1.000



Качество ухудшилось, посмотрим на примеры токенизации:

In [50]:
samples = train_data[:3]
samples

[['converting string to list',
  'Convert Google results object (pure js) to Python object'],
 ['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?',
  'Event handling for geometries in Three.js?'],
 ['Sending array via Ajax fails',
  'Getting all list items of an unordered list in PHP']]

In [51]:
print("default tokenizer:", list(map(lambda x: tokenizer(' '.join(x)), samples)), sep='\n', end='\n\n')
print("nltk tokenizer:", list(map(lambda x: word_tokenize(' '.join(x)), samples)), sep='\n')

default tokenizer:
[['converting', 'string', 'to', 'list', 'Convert', 'Google', 'results', 'object', 'pure', 'js', 'to', 'Python', 'object'], ['Which', 'HTML', '5', 'Canvas', 'Javascript', 'to', 'use', 'for', 'making', 'an', 'interactive', 'drawing', 'tool', 'Event', 'handling', 'for', 'geometries', 'in', 'Three', 'js'], ['Sending', 'array', 'via', 'Ajax', 'fails', 'Getting', 'all', 'list', 'items', 'of', 'an', 'unordered', 'list', 'in', 'PHP']]

nltk tokenizer:
[['converting', 'string', 'to', 'list', 'Convert', 'Google', 'results', 'object', '(', 'pure', 'js', ')', 'to', 'Python', 'object'], ['Which', 'HTML', '5', 'Canvas', 'Javascript', 'to', 'use', 'for', 'making', 'an', 'interactive', 'drawing', 'tool', '?', 'Event', 'handling', 'for', 'geometries', 'in', 'Three.js', '?'], ['Sending', 'array', 'via', 'Ajax', 'fails', 'Getting', 'all', 'list', 'items', 'of', 'an', 'unordered', 'list', 'in', 'PHP']]


Токенизатор из `nltk` не отсеивает пунктуацию. Конечно, он и не должен. На деле наш токенизатор не является таковым, объединяя несколько шагов предобработки.


Ухудшение метрик при использовании `nltk`-токенизатора в случае обучения собственных эмбеддингов может быть связано с уменьшением количества слов попадающих в одно окно: между словам остаются токены пунктуации.

In [52]:
sentence = "There are a lot of popular & reliable languages, e.g. C++, C#, Python. Which should I learn as the 1st?"
print("default tokenizer:", tokenizer(sentence), sep='\n', end='\n\n')
print("nltk tokenizer:", word_tokenize(sentence), sep='\n')

default tokenizer:
['There', 'are', 'a', 'lot', 'of', 'popular', 'reliable', 'languages', 'e', 'g', 'C', 'C', 'Python', 'Which', 'should', 'I', 'learn', 'as', 'the', '1st']

nltk tokenizer:
['There', 'are', 'a', 'lot', 'of', 'popular', '&', 'reliable', 'languages', ',', 'e.g', '.', 'C++', ',', 'C', '#', ',', 'Python', '.', 'Which', 'should', 'I', 'learn', 'as', 'the', '1st', '?']


Наш токенизатор, однако, не ловит специфичные слова, содержащие небуквенные символы: С++, C#, Three.js.
Впрочем, `nltk`-токенизатор тоже не определяет некоторые из них, например, C#. 

In [53]:
from string import printable, ascii_letters, digits, punctuation, whitespace

set(printable) == set(ascii_letters + digits + punctuation + whitespace)

True

In [54]:
print(*list(punctuation))
print(*[repr(ch) for ch in whitespace])

! " # $ % & ' ( ) * + , - . / : ; < = > ? @ [ \ ] ^ _ ` { | } ~
' ' '\t' '\n' '\r' '\x0b' '\x0c'


In [55]:
print("Tokenization of words with some special chars within by word_tokenize from nltk:", end='\n\n')
for char_set, char_desc in [(punctuation, 'punctuation'), (whitespace, 'whitespace')]:
    print(char_desc, '-' * len(char_desc), sep='\n', end='\n\n')
    generated_words = [ch + 'before' for ch in char_set]
    generated_words += ['with' + ch + 'in' for ch in char_set]
    generated_words += ['after' + ch for ch in char_set]
    tokenized = [word_tokenize(w) for w in generated_words]
    print("generated words:", generated_words, end='\n\n---\n\n')
    print("single token:", [tokens for tokens in tokenized if len(tokens) == 1], end='\n\n---\n\n')
    print("splitted:", [tokens for tokens in tokenized if len(tokens) > 1], end='\n\n')

Tokenization of words with some special chars within by word_tokenize from nltk:

punctuation
-----------

generated words: ['!before', '"before', '#before', '$before', '%before', '&before', "'before", '(before', ')before', '*before', '+before', ',before', '-before', '.before', '/before', ':before', ';before', '<before', '=before', '>before', '?before', '@before', '[before', '\\before', ']before', '^before', '_before', '`before', '{before', '|before', '}before', '~before', 'with!in', 'with"in', 'with#in', 'with$in', 'with%in', 'with&in', "with'in", 'with(in', 'with)in', 'with*in', 'with+in', 'with,in', 'with-in', 'with.in', 'with/in', 'with:in', 'with;in', 'with<in', 'with=in', 'with>in', 'with?in', 'with@in', 'with[in', 'with\\in', 'with]in', 'with^in', 'with_in', 'with`in', 'with{in', 'with|in', 'with}in', 'with~in', 'after!', 'after"', 'after#', 'after$', 'after%', 'after&', "after'", 'after(', 'after)', 'after*', 'after+', 'after,', 'after-', 'after.', 'after/', 'after:', 'after;',

In [56]:
print("stackoverflow embeddings vocab examples:", soe_vocab_examples, sep='\n', end='\n\n')
tokenized_by_nltk = [word_tokenize(w) for w in soe_vocab_examples]
print("their tokenization by nltk (only which splitted):", [tokens for tokens in tokenized_by_nltk if len(tokens) > 1], sep='\n')

stackoverflow embeddings vocab examples:
['use', 'diplayed', 'udpated', 'phpcr', '75x', 'unviewable', 'com/it-it/library/system', 'not-recommended', 'another_user', 'regusers', 'com/components/#dropdowns', 'rmiinterface', 'updatemask', '50+50', 'onfileopen', 'tophone', '9193', 'appusermodelids', 'ÑÐºÐ', '_levels', 'programmer/user', 'greenrectangle', 'intar', 'casescontroller', 'com/docs/security/', 'trasnaction', '/app-root/runtime/repo', 'nav-direction', 'jaredc', 'start+duration', 'com/en-us/sql/t-sql/language-elements/case-transact-sql', 'e103', 'anyway/', 'staa', 'limitedness', 'draw_size', 'com/border-image', 'art_sifra', 'xmlaolap4jutil', 'uibuttion', 'domagoj', 'check/apply', '10994058', 'make-account', 'yourmode', 'type/record', '823x776', 'somethod', 'num_order', '9/20/2014', 'filter14']

their tokenization by nltk (only which splitted):
[['com/components/', '#', 'dropdowns']]


In [57]:
print(word_tokenize("hello.. world... more dots...... etc., e.g. etc.: aaaa!!! bbb!! cc?? C# C/C++"))

['hello', '..', 'world', '...', 'more', 'dots', '......', 'etc.', ',', 'e.g', '.', 'etc', '.', ':', 'aaaa', '!', '!', '!', 'bbb', '!', '!', 'cc', '?', '?', 'C', '#', 'C/C++']


In [58]:
print("Punctuation chars as words in the stackoverflow embeddings vocab:", [ch for ch in punctuation if ch in so_embeddings])
print("Whitespace chars as words in the stackoverflow embeddings vocab:", [ch for ch in whitespace if ch in so_embeddings])

Punctuation chars as words in the stackoverflow embeddings vocab: ['#', '+', '-', '/', '\\', '_']
Whitespace chars as words in the stackoverflow embeddings vocab: []


При этом токены спецсимволов мы не отсеиваем и некоторые из них присутствуют в словаре предобученных эмбеддингов, что, вероятно, дает некоторый "шум", с чем и может быть связано ухудшение на предобученных эмбеддингах. 

In [59]:
print(
    f"{'C#' in so_embeddings = }",
    f"{'C#'.lower() in so_embeddings = }",
    f"{'C#'.lower() in so_embeddings = }",
    f"{'Three.js' in so_embeddings = }",
    f"{'Three.js'.lower() in so_embeddings = }",
    f"{'.js' in so_embeddings = }",
    f"{'js' in so_embeddings = }",
    f"{'.json' in so_embeddings = }",
    f"{'helloworld' in so_embeddings = }",
    f"{'helloworld.py' in so_embeddings = }",
    f"{'helloworld.cpp' in so_embeddings = }",
    f"{'.cpp' in so_embeddings = }",
    f"{'cpp' in so_embeddings = }",
    sep='\n')

'C#' in so_embeddings = False
'C#'.lower() in so_embeddings = True
'C#'.lower() in so_embeddings = True
'Three.js' in so_embeddings = False
'Three.js'.lower() in so_embeddings = False
'.js' in so_embeddings = False
'js' in so_embeddings = True
'.json' in so_embeddings = False
'helloworld' in so_embeddings = True
'helloworld.py' in so_embeddings = False
'helloworld.cpp' in so_embeddings = False
'.cpp' in so_embeddings = False
'cpp' in so_embeddings = True


In [60]:
for word in so_embeddings.vocab:
    if '.' in word:
        print("Example of a word with a dot within in the stackoverflow embeddings vocab:", word)
        break
else:
    print("There is no dot within words in the stackoverflow embeddings vocab")

There is no dot within words in the stackoverflow embeddings vocab


In [61]:
soe_chars_within_tokens = set()
for word in so_embeddings.vocab:
        soe_chars_within_tokens.update(ch for ch in punctuation if ch in word and ch != word)
print("Punctuation chars within words in the stackoverflow embeddings vocab:", *soe_chars_within_tokens)

Punctuation chars within words in the stackoverflow embeddings vocab: _ \ + # / -


Кроме того, `nltk`-токенизатор определяет как единый токен такие сочетания, как Three.js, helloworld.cpp, но они не присутствуют в предобученных эмбеддингах, т.е. методы токенизации не согласованы.

Просматривающиеся пути улучшения качества:
- доработка собственного токенизатора добавлением учета спецсимволов в регулярное выражение
- добавление постпроцессинга для `nltk`-токенизатора: отсев токенов пунтуации, разбиение токенов наподобие helloworld.cpp, объединение некоторых последовательных токенов, подобных C и #. Последнее проблематично, ибо это обратное преобразование, не являющееся однозначным: спецсимвол мог находится в начале слова, в середине или конце.

## Experiment #7:
improved tokenizer + lowercase & stackoverflow embeddings

In [62]:
import re
from string import punctuation


soe_chars_within_tokens = "-_#/+\\"
multi_char_punct_mark_chars = ['.'] 
word_token_chars = ''.join([re.escape(ch) for ch in soe_chars_within_tokens]) + r"\w"
punct_multi_chars = ''.join([re.escape(ch) for ch in multi_char_punct_mark_chars])
punct_single_chars = ''.join([re.escape(ch) for ch in punctuation])
any_non_whitespace_chars = r"^\s"
token_pattern = re.compile(
    f"[{word_token_chars}]+|"
    f"[{punct_multi_chars}]+|"
    f"[{punct_single_chars}]|"
    f"[{any_non_whitespace_chars}]+"
    )


def improved_tokenizer(text):
    return token_pattern.findall(text)


def filter_punctuation(tokens, punctuation_chars=punctuation):
    for token in tokens:
        if any(ch not in punctuation_chars for ch in token):
            yield token

In [63]:
test_text = "/asdf/jkl C# dir/ Windows\\System32 c/c++ .cpp ... ?: e.g. --enable helloworld.py\tjohn\ndoe Салтыков-Щедрин"

In [64]:
print(improved_tokenizer(test_text))

['/asdf/jkl', 'C#', 'dir/', 'Windows\\System32', 'c/c++', '.', 'cpp', '...', '?', ':', 'e', '.', 'g', '.', '--enable', 'helloworld', '.', 'py', 'john', 'doe', 'Салтыков-Щедрин']


In [65]:
def preprocess(data):
    return list(filter_punctuation(improved_tokenizer(data.lower())))

In [66]:
print(preprocess(test_text))

['/asdf/jkl', 'c#', 'dir/', 'windows\\system32', 'c/c++', 'cpp', 'e', 'g', '--enable', 'helloworld', 'py', 'john', 'doe', 'салтыков-щедрин']


In [67]:
soe_based_ranking = make_validation(so_embeddings, preprocess, validation_data)

In [68]:
metrics['experiment'] = "#7: improved tokenizer + lowercase & stackoverflow embds"
metrics.update(get_metrics(soe_based_ranking))

DCG@   1: 0.411 | Hits@   1: 0.411
DCG@   5: 0.495 | Hits@   5: 0.573
DCG@  10: 0.516 | Hits@  10: 0.638
DCG@ 100: 0.561 | Hits@ 100: 0.864
DCG@ 500: 0.576 | Hits@ 500: 0.971
DCG@1000: 0.579 | Hits@1000: 1.000



Немного не дотянули до результатов с простейшим токенизатором + lowercase на предобученных эмбеддингах (эксперимент #3). **Почему?**

## Experiment #8:
improved tokenizer + lowercase & our own embeddings (default window (5), default min_count (5))

In [69]:
%%time
ex_8_embeddings_trained = train_embeddings(prepare_data(train_data, preprocess), window=5, min_count=5)

Wall time: 42.5 s


In [70]:
our_embeddings_based_ranking = make_validation(ex_8_embeddings_trained, preprocess, validation_data)

In [71]:
metrics['experiment'] = "#8: improved tokenizer + lowercase & our own embds (win 5, min_cnt 5)"
metrics.update(get_metrics(our_embeddings_based_ranking))

DCG@   1: 0.250 | Hits@   1: 0.250
DCG@   5: 0.304 | Hits@   5: 0.351
DCG@  10: 0.320 | Hits@  10: 0.402
DCG@ 100: 0.358 | Hits@ 100: 0.589
DCG@ 500: 0.390 | Hits@ 500: 0.838
DCG@1000: 0.407 | Hits@1000: 1.000



Аналогично, наблюдается ухудшение качества по сравнению с простейшим токенизатором + lowercase на наших собственных эмбеддингах (эксперимент #4). **Почему?**

## Experiment #9:
word_tokenize from nltk + lowercase + postprocess (punctuation filter, split by dot) & stackoverflow embeddings

In [72]:
from nltk.tokenize import word_tokenize


SEPARATORS = "."
split_pattern = re.compile('|'.join(re.escape(ch) for ch in SEPARATORS))


def split_words(words_list, separators=SEPARATORS):
    result = []
    for word in words_list:
        result.extend(split_pattern.split(word))
    return result


def preprocess(data):
    return list(filter_punctuation(split_words(word_tokenize(data.lower()))))

In [73]:
soe_based_ranking = make_validation(so_embeddings, preprocess, validation_data)

In [74]:
metrics['experiment'] = "#9: word_tokenize from nltk + lowercase + punct filter + split by dot & stackoverflow embds"
metrics.update(get_metrics(soe_based_ranking)) 

DCG@   1: 0.408 | Hits@   1: 0.408
DCG@   5: 0.493 | Hits@   5: 0.573
DCG@  10: 0.514 | Hits@  10: 0.636
DCG@ 100: 0.560 | Hits@ 100: 0.862
DCG@ 500: 0.574 | Hits@ 500: 0.970
DCG@1000: 0.577 | Hits@1000: 1.000



Токенизатор из `nltk` по-прежнему работает хуже самописного (и default (#3), и improved (#7)) на предобученных эмбеддингах, даже вместе отсеиванием пунктуации и разбиением по точке, но по сравнению с его предыдущим результатом (эксперимент #5) результат улучшился.

## Experiment #10:
word_tokenize from nltk + lowercase + postprocess (punctuation filter, split by dot) & our own embeddings (default window (5), default min_count (5))

In [75]:
%%time
ex_10_embeddings_trained = train_embeddings(prepare_data(train_data, preprocess), window=5, min_count=5)

Wall time: 4min 46s


In [76]:
our_embeddings_based_ranking = make_validation(ex_10_embeddings_trained, preprocess, validation_data)

In [77]:
metrics['experiment'] = "#10: word_tokenize from nltk + lowercase + punct filter + split by dot & our own embds (win 5, min_cnt 5)"
metrics.update(get_metrics(our_embeddings_based_ranking))

DCG@   1: 0.254 | Hits@   1: 0.254
DCG@   5: 0.304 | Hits@   5: 0.347
DCG@  10: 0.323 | Hits@  10: 0.405
DCG@ 100: 0.360 | Hits@ 100: 0.592
DCG@ 500: 0.391 | Hits@ 500: 0.840
DCG@1000: 0.408 | Hits@1000: 1.000



На наших эмбеддингах вместе с отсеиванием пунктуации и разбиением по точке `nltk`-токенизатор улучшил свои результаты (эксперимент #6), а также побил/чуть уступил improved tokenizer из #8 \[не всегда это получается, зависит от seed], но не дотянулся до default tokenizer (#4).

## Default tokenizer VS improved tokenizer
Попробум разобраться в подобных результатах работы улучшенного токенизатора, сравнив его с исходным по рангу дубликатов на валидационных данных. Для начала с использованием предобученных эмбеддингов, при использовании которых с улучшенным токенизатором качество упало с 0.415 (#3) до 0.411 (#7) \[результат одинаков на паре прогонов].

In [78]:
def compare_tokenizers_using_validation(embeddings, tokenizer1, tokenizer2, validation_data, max_validation_examples=MAX_VAL_EX):
    val_data_inds_on_which_diff_rank = []
    for i, line in tqdm(
            enumerate(islice(validation_data, max_validation_examples)),
            total=max_validation_examples):
        q, *ex = line
        ranks1 = rank_candidates(q, ex, embeddings, tokenizer1)
        ranking1 = [r[0] for r in ranks1].index(0) + 1
        ranks2 = rank_candidates(q, ex, embeddings, tokenizer2)
        ranking2 = [r[0] for r in ranks2].index(0) + 1
        if ranking1 != ranking2:
            val_data_inds_on_which_diff_rank.append((i, (ranking1, ranking2)))
    return val_data_inds_on_which_diff_rank

In [79]:
def preprocess_with_default_tokenizer(data):  # expr #3
    return tokenizer(data.lower())

def preprocess_with_improved_tokenizer(data):  # expr 7 
    return list(filter_punctuation(improved_tokenizer(data.lower())))

In [80]:
val_data_inds_on_which_diff_rank = compare_tokenizers_using_validation(so_embeddings, preprocess_with_default_tokenizer, preprocess_with_improved_tokenizer, validation_data)

In [81]:
len(val_data_inds_on_which_diff_rank)

405

In [82]:
def show_tokenization_result(val_data_inds_on_which_diff_rank, tok1, tok2, tok1_name="tok1", tok2_name="tok2", min_rank_diff=-1, max_q_from_one_val=5):
    for val_data_i, (tok1_rank, tok2_rank) in val_data_inds_on_which_diff_rank:
        rank_diff = tok1_rank - tok2_rank
        if ((min_rank_diff > 0) & (rank_diff >= min_rank_diff)) or ((min_rank_diff < 0) & (rank_diff <= min_rank_diff)):
            if rank_diff > 0:
                w1, w2 = "worse", "better"
            else:
                w1, w2 = "better", "worse"
            q_counter = 0
            for i, q in enumerate(validation_data[val_data_i]):
                t1_res = tok1(q)
                t2_res = tok2(q)
                if t1_res != t2_res:
                    q_counter += 1
                    if q_counter > max_q_from_one_val:
                        break 
                    print(f"{tok1_name} ({w1} res) & {tok2_name} ({w2} res) on {i} query from val data #{val_data_i}:", t1_res,t2_res, sep='\n', end='\n\n')

### default tokenizer is *better* (rank less) than improved tokenizer

In [83]:
MIN_RANK_DIFF = -50
MAX_QUERIES_FROM_ONE_VAL_SAMPLE = 5
show_tokenization_result(val_data_inds_on_which_diff_rank, preprocess_with_default_tokenizer, preprocess_with_improved_tokenizer, "default tokenizer", "improved tokenizer", min_rank_diff=MIN_RANK_DIFF, max_q_from_one_val=MAX_QUERIES_FROM_ONE_VAL_SAMPLE)

ny2']

default tokenizer (better res) & improved tokenizer (worse res) on 21 query from val data #390:
['f', 'nativeptr', 'stackalloc', 'unexpected', 'stack', 'overflow']
['f#', 'nativeptr', 'stackalloc', 'unexpected', 'stack', 'overflow']

default tokenizer (better res) & improved tokenizer (worse res) on 0 query from val data #395:
['android', 'de', 'facto', 'implementation', 'of', 'rest', 'json', 'client']
['android', 'de-facto', 'implementation', 'of', 'rest/json', 'client']

default tokenizer (better res) & improved tokenizer (worse res) on 4 query from val data #395:
['insert', 'close', 'row', 'after', 'every', '4', 'in', 'foreach', 'loop']
['insert/close', 'row', 'after', 'every', '4', 'in', 'foreach', 'loop']

default tokenizer (better res) & improved tokenizer (worse res) on 5 query from val data #395:
['use', 'internal', 'static', 'class', 'methods', 'from', 'external', 'dll', 'using', 'c']
['use', 'internal', 'static', 'class', 'methods', 'from', 'external', 'dll', 'using', 

### default tokenizer is *worse* (rank higher) than improved tokenizer

In [84]:
MIN_RANK_DIFF = 50
show_tokenization_result(val_data_inds_on_which_diff_rank, preprocess_with_default_tokenizer, preprocess_with_improved_tokenizer, "default tokenizer", "improved tokenizer", min_rank_diff=MIN_RANK_DIFF, max_q_from_one_val=MAX_QUERIES_FROM_ONE_VAL_SAMPLE)

default tokenizer (worse res) & improved tokenizer (better res) on 0 query from val data #12:
['hierarchy', 'parent', 'child', 'relationship', 'datagrid', 'c', 'window', 'application']
['hierarchy', 'parent', 'child', 'relationship', 'datagrid', 'c#', 'window', 'application']

default tokenizer (worse res) & improved tokenizer (better res) on 1 query from val data #12:
['expanding', 'collapsing', 'in', 'datagridview']
['expanding/collapsing', 'in', 'datagridview']

default tokenizer (worse res) & improved tokenizer (better res) on 19 query from val data #12:
['unable', 'to', 'run', 'rails', 'server', 'require', 'cannot', 'load', 'such', 'file', 'net', 'ssh', 'loaderror']
['unable', 'to', 'run', 'rails', 'server', 'require', 'cannot', 'load', 'such', 'file', 'net/ssh', 'loaderror']

default tokenizer (worse res) & improved tokenizer (better res) on 21 query from val data #12:
['how', 'to', 'error', 'handle', 'overclicking', 'in', 'c']
['how', 'to', 'error', 'handle', 'overclicking', 'in

Между случаями, когда один лучше другого, какого-то значимого различия не наблюдается, но можно попробовать бить слова по forward slash и hyphen. 

## Experiment #11:
improved tokenizer v2 (without forward slash & hyphen) + lowercase & stackoverflow embeddings


In [85]:
import re
from string import punctuation


soe_chars_within_tokens_without_forward_slash_and_hyphen = "_#+\\"
word_token_chars_v2 = ''.join([re.escape(ch) for ch in soe_chars_within_tokens_without_forward_slash_and_hyphen]) + r"\w"
token_pattern_v2 = re.compile(
    f"[{word_token_chars_v2}]+|"
    f"[{punct_multi_chars}]+|"
    f"[{punct_single_chars}]|"
    f"[{any_non_whitespace_chars}]+"
    )


def improved_tokenizer_v2(text):
    return token_pattern_v2.findall(text)

In [86]:
def preprocess(data):
    return list(filter_punctuation(improved_tokenizer_v2(data.lower())))

In [87]:
soe_based_ranking = make_validation(so_embeddings, preprocess, validation_data)

In [88]:
metrics['experiment'] = "#11: imprvd tok v2 (wo forward slash & hyphen) + lowercase & & stackoverflow embds"
metrics.update(get_metrics(soe_based_ranking))

DCG@   1: 0.418 | Hits@   1: 0.418
DCG@   5: 0.504 | Hits@   5: 0.584
DCG@  10: 0.525 | Hits@  10: 0.650
DCG@ 100: 0.571 | Hits@ 100: 0.877
DCG@ 500: 0.584 | Hits@ 500: 0.974
DCG@1000: 0.587 | Hits@1000: 1.000



## Experiment #12:
improved tokenizer v2 (without forward slash & hyphen) + lowercase & our own embeddings (default window (5), default min_count (5))

In [89]:
%%time
ex_12_embeddings_trained = train_embeddings(prepare_data(train_data, preprocess), window=5, min_count=5)

Wall time: 41.7 s


In [90]:
our_embeddings_based_ranking = make_validation(ex_12_embeddings_trained, preprocess, validation_data)

In [91]:
metrics['experiment'] = "#12: imprvd tok v2 (wo forward slash & hyphen) + lowercase & our own embds (win 5, min_cnt 5)"
metrics.update(get_metrics(our_embeddings_based_ranking))

DCG@   1: 0.261 | Hits@   1: 0.261
DCG@   5: 0.313 | Hits@   5: 0.358
DCG@  10: 0.330 | Hits@  10: 0.412
DCG@ 100: 0.369 | Hits@ 100: 0.604
DCG@ 500: 0.400 | Hits@ 500: 0.845
DCG@1000: 0.416 | Hits@1000: 1.000



Ура-ура, и на предобученном, и на нашем эмбеддингах побиты \[на наших зависит от прогона] результаты дефолтного простейшего токенизатора. Видимо, на наших данных всё же важнее иметь больших отдельных смысловых единц (части составных слов), т.к., вероятно, мало таких слов, смысл которых не выводится из этих отдельных частей.

# Stop words

## Experiment #13:
improved tokenizer v2 + lowercase + removing stop words & stackoverflow embeddings


In [92]:
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def remove_stop_words(words, stop_words=stop_words):
    for word in words:
        if word not in stop_words:
            yield word

def preprocess(data):
    return list(remove_stop_words(filter_punctuation(improved_tokenizer_v2(data.lower()))))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user0\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [93]:
soe_based_ranking = make_validation(so_embeddings, preprocess, validation_data)

In [94]:
metrics['experiment'] = "#13: imprvd tok v2 + lowercase + rmv stp wrds & stackoverflow embds"
metrics.update(get_metrics(soe_based_ranking))

DCG@   1: 0.418 | Hits@   1: 0.418
DCG@   5: 0.504 | Hits@   5: 0.583
DCG@  10: 0.525 | Hits@  10: 0.650
DCG@ 100: 0.572 | Hits@ 100: 0.878
DCG@ 500: 0.584 | Hits@ 500: 0.974
DCG@1000: 0.587 | Hits@1000: 1.000



In [95]:
print(f"{len(stop_words) = }")
print("stop words in stackoverflow embeddings:", [word for word in stop_words if word in so_embeddings])

len(stop_words) = 179
stop words in stackoverflow embeddings: ['ours', 'himself', 'will', 'just', 'now', 'o', 'y', 'ain', 'ma', 'mightn', 'needn']


Качество на предобученных эмбеддингах не изменилось, поскольку они и так почти не содержат стоп-слов.

## Experiment #14:
improved tokenizer v2 + lowercase + removing stop words & our own embeddings (default window (5), default min_count (5))

In [96]:
%%time
ex_14_embeddings_trained = train_embeddings(prepare_data(train_data, preprocess), window=5, min_count=5)

Wall time: 1min 29s


In [97]:
our_embeddings_based_ranking = make_validation(ex_14_embeddings_trained, preprocess, validation_data)

In [98]:
metrics['experiment'] = "#14: imprvd tok v2 + lowercase + rmv stp wrds & our own embds (win 5, min_cnt 5)"
metrics.update(get_metrics(our_embeddings_based_ranking))

DCG@   1: 0.367 | Hits@   1: 0.367
DCG@   5: 0.438 | Hits@   5: 0.501
DCG@  10: 0.455 | Hits@  10: 0.555
DCG@ 100: 0.487 | Hits@ 100: 0.714
DCG@ 500: 0.504 | Hits@ 500: 0.845
DCG@1000: 0.520 | Hits@1000: 1.000



Качество на наших эмбеддингах после удаления стоп-слов очень сильно выросло (пред. макс. рез. в #12): больше значащих слов попадает в окно при обучении, а при усреднении векторов слов предложения смысл не смазывается практически ничего не значащими в силу их повсеместности стоп-словами.

# Stemming

## Experiment #15:
improved tokenizer v2 + lowercase + removing stop words + PorterStemmer & stackoverflow embeddings


In [99]:
from nltk.stem import PorterStemmer


ps = PorterStemmer()


def get_stem(words, stemmer=ps.stem):
    for word in words:
        yield stemmer(word)


def preprocess(data):
    return list(get_stem(remove_stop_words(filter_punctuation(improved_tokenizer_v2(data.lower())))))

In [100]:
soe_based_ranking = make_validation(so_embeddings, preprocess, validation_data)

In [101]:
metrics['experiment'] = "#15: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & stackoverflow embds"
metrics.update(get_metrics(soe_based_ranking))

DCG@   1: 0.323 | Hits@   1: 0.323
DCG@   5: 0.402 | Hits@   5: 0.474
DCG@  10: 0.423 | Hits@  10: 0.541
DCG@ 100: 0.475 | Hits@ 100: 0.797
DCG@ 500: 0.495 | Hits@ 500: 0.948
DCG@1000: 0.500 | Hits@1000: 1.000



Качество на предобученных эмбеддингах сильно упало, потому что они обучены на целых словах, а не их основах.

## Experiment #16:
improved tokenizer v2 + lowercase + removing stop words + PorterStemmer & our own embeddings (default window (5), default min_count (5))

In [102]:
%%time
ex_16_embeddings_trained = train_embeddings(prepare_data(train_data, preprocess), window=5, min_count=5)

Wall time: 7min 50s


In [103]:
our_embeddings_based_ranking = make_validation(ex_16_embeddings_trained, preprocess, validation_data)

In [104]:
metrics['experiment'] = "#16: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & our own embds (win 5, min_cnt 5)"
metrics.update(get_metrics(our_embeddings_based_ranking))

DCG@   1: 0.397 | Hits@   1: 0.397
DCG@   5: 0.479 | Hits@   5: 0.547
DCG@  10: 0.496 | Hits@  10: 0.601
DCG@ 100: 0.532 | Hits@ 100: 0.777
DCG@ 500: 0.546 | Hits@ 500: 0.880
DCG@1000: 0.558 | Hits@1000: 1.000



Качество на наших эмбеддингах выросло (по сравнению с #14). Беря основы слов, мы теряем часть информации, а для некоторых слов они вообще могут быть ошибочны, но, видимо, для Word2Vec увеличенная частота слова, а значит и встречаемость в разных контекстах важнее для выделения смысла, по крайней мере, на малых корпусах.

# Lemmatization

## Experiment #17:
improved tokenizer v2 + lowercase + removing stop words + WordNetLemmatizer & stackoverflow embeddings


In [105]:
import nltk

nltk.download('wordnet')

from nltk import WordNetLemmatizer

wnl = WordNetLemmatizer()

def get_lemma(words, lemmatizer=wnl.lemmatize):
    for word in words:
        yield lemmatizer(word)

def preprocess(data):
    return list(get_lemma(remove_stop_words(filter_punctuation(improved_tokenizer_v2(data.lower())))))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user0\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [106]:
soe_based_ranking = make_validation(so_embeddings, preprocess, validation_data)

In [107]:
metrics['experiment'] = "#17: imprvd tok v2 + lowercase + rmv stp wrds + WordNetLemmatizer & stackoverflow embds"
metrics.update(get_metrics(soe_based_ranking))

DCG@   1: 0.409 | Hits@   1: 0.409
DCG@   5: 0.495 | Hits@   5: 0.573
DCG@  10: 0.515 | Hits@  10: 0.636
DCG@ 100: 0.564 | Hits@ 100: 0.872
DCG@ 500: 0.576 | Hits@ 500: 0.967
DCG@1000: 0.580 | Hits@1000: 1.000



Качество немного снизилось по сравнению с #13. Причина, вероятно, та же, что и в #15: эмбеддинги обучались на ненормализованных словах и потому нормализации данных только мешает. Падение качества не столь сильное, как со стеммингом, поскольку лемматизация, очевидно, не так сильно меняет слова.

## Experiment #18:
improved tokenizer v2 + lowercase + removing stop words + WordNetLemmatizer & our own embeddings (default window (5), default min_count (5))

In [108]:
%%time
ex_18_embeddings_trained = train_embeddings(prepare_data(train_data, preprocess), window=5, min_count=5)

Wall time: 2min 51s


In [109]:
our_embeddings_based_ranking = make_validation(ex_18_embeddings_trained, preprocess, validation_data)

In [110]:
metrics['experiment'] = "#18: imprvd tok v2 + lowercase + rmv stp wrds + WordNetLemmatizer & our own embds (win 5, min_cnt 5)"
metrics.update(get_metrics(our_embeddings_based_ranking))

DCG@   1: 0.390 | Hits@   1: 0.390
DCG@   5: 0.456 | Hits@   5: 0.517
DCG@  10: 0.474 | Hits@  10: 0.572
DCG@ 100: 0.510 | Hits@ 100: 0.750
DCG@ 500: 0.523 | Hits@ 500: 0.856
DCG@1000: 0.539 | Hits@1000: 1.000



При обучении собственных эмбеддингов нормализация в виде лемматизации помогает: результат лучше максимального без какой-либо нормализации (эксперимент #14). Но на наших данных нормализация в виде стемминга работает ещё лучше (#16).

## Experiment #19:
improved tokenizer v2 + lowercase + removing stop words + PorterStemmer & our own embeddings (window 2, min_count 5)

In [111]:
def preprocess(data):
    return list(get_stem(remove_stop_words(filter_punctuation(improved_tokenizer_v2(data.lower())))))

In [112]:
%%time
ex_19_embeddings_trained = train_embeddings(prepare_data(train_data, preprocess), window=2, min_count=5)

Wall time: 7min 29s


In [113]:
our_embeddings_based_ranking = make_validation(ex_19_embeddings_trained, preprocess, validation_data)

In [114]:
metrics['experiment'] = "#19: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & our own embds (win 2, min_cnt 5)"
metrics.update(get_metrics(our_embeddings_based_ranking))

DCG@   1: 0.397 | Hits@   1: 0.397
DCG@   5: 0.479 | Hits@   5: 0.547
DCG@  10: 0.496 | Hits@  10: 0.601
DCG@ 100: 0.532 | Hits@ 100: 0.777
DCG@ 500: 0.546 | Hits@ 500: 0.880
DCG@1000: 0.558 | Hits@1000: 1.000



Уменьшение окна практически не изменило качество, что логично, ведь если слово определяется контекстом и неинформативные (стоп-слова) удалены, то наиболее важные и определяющие слова будут находится рядом с определяемым словом.

## Experiment #20:
improved tokenizer v2 + lowercase + removing stop words + PorterStemmer & our own embeddings (window 10, min_count 5)

In [115]:
%%time
ex_20_embeddings_trained = train_embeddings(prepare_data(train_data, preprocess), window=10, min_count=5)

Wall time: 7min 30s


In [116]:
our_embeddings_based_ranking = make_validation(ex_20_embeddings_trained, preprocess, validation_data)

In [117]:
metrics['experiment'] = "#20: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & our own embds (win 10, min_cnt 5)"
metrics.update(get_metrics(our_embeddings_based_ranking))

DCG@   1: 0.397 | Hits@   1: 0.397
DCG@   5: 0.479 | Hits@   5: 0.547
DCG@  10: 0.496 | Hits@  10: 0.601
DCG@ 100: 0.532 | Hits@ 100: 0.777
DCG@ 500: 0.546 | Hits@ 500: 0.880
DCG@1000: 0.558 | Hits@1000: 1.000



Увеличение окна также не изменило качество. Видимо, причина в том, что сами документы у нас очень короткие и даже с окном в 5 (захватывает до 10 слов) общий контекст слова был близок ко всему документу (вопросу).

## Experiment #21:
improved tokenizer v2 + lowercase + removing stop words + PorterStemmer & our own embeddings (window 5, min_count 10)

In [118]:
%%time
ex_21_embeddings_trained = train_embeddings(prepare_data(train_data, preprocess), window=5, min_count=10)

Wall time: 7min 25s


In [119]:
our_embeddings_based_ranking = make_validation(ex_21_embeddings_trained, preprocess, validation_data)

In [120]:
metrics['experiment'] = "#21: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & our own embds (win 5, min_cnt 10)"
metrics.update(get_metrics(our_embeddings_based_ranking))

DCG@   1: 0.392 | Hits@   1: 0.392
DCG@   5: 0.473 | Hits@   5: 0.540
DCG@  10: 0.492 | Hits@  10: 0.598
DCG@ 100: 0.528 | Hits@ 100: 0.773
DCG@ 500: 0.541 | Hits@ 500: 0.881
DCG@1000: 0.554 | Hits@1000: 1.000



In [121]:
len(train_data)

1000000

Интересно, что требование встречаемости слова в хотя бы 0.001% документов вместо 0.0005% немного, но уменьшает качество. Но если учесть, что мы ищем дубликаты вопросов, то это логично: подобные уникальные слова хорошо эти дубликаты выделяют.

## Experiment #22:
improved tokenizer v2 + lowercase + removing stop words + PorterStemmer & our own embeddings (window 5, min_count 2)

In [122]:
%%time
ex_22_embeddings_trained = train_embeddings(prepare_data(train_data, preprocess), window=5, min_count=2)

Wall time: 7min 41s


In [123]:
our_embeddings_based_ranking = make_validation(ex_22_embeddings_trained, preprocess, validation_data)

In [124]:
metrics['experiment'] = "#22: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & our own embds (win 5, min_cnt 2)"
metrics.update(get_metrics(our_embeddings_based_ranking))

DCG@   1: 0.400 | Hits@   1: 0.400
DCG@   5: 0.483 | Hits@   5: 0.554
DCG@  10: 0.501 | Hits@  10: 0.607
DCG@ 100: 0.536 | Hits@ 100: 0.779
DCG@ 500: 0.549 | Hits@ 500: 0.882
DCG@1000: 0.561 | Hits@1000: 1.000



Здесь качество по сравнению с `min_count=5` немного выросло, что подтвержает предыдущее предположение про полезность уникальных слов для данной конкретной задачи.

## Results comparison
Для наглядности даны 4 таблицы: полная таблица экспериментов; она же, но отсортированная по $DCG@1$; по отдельной таблице только с экспериментами на предобученных эмбеддингах и на наших собственных эмбеддингах, обе отсортированы по $DCG@1$.

### Complete table of experiments

In [125]:
import pandas as pd

df = pd.DataFrame(metrics)
df.set_index('experiment', inplace=True)
df

,DCG@1,Hits@1,DCG@5,Hits@5,DCG@10,Hits@10,DCG@100,Hits@100,DCG@500,Hits@500,DCG@1000,Hits@1000
experiment,,,,,,,,,,,,
#1: def tknz & stackoverflow embds,0.285,0.285,0.341622,0.393,0.359693,0.449,0.405681,0.679,0.431275,0.879,0.443923,1.0
"#2: def tknz & our own embds (win 5, min_cnt 5)",0.198,0.198,0.237854,0.274,0.250290,0.313,0.290426,0.516,0.324155,0.783,0.346984,1.0
#3: def tknz + lowercase & stackoverflow embds,0.415,0.415,0.502495,0.582,0.524731,0.651,0.570256,0.874,0.583425,0.973,0.586335,1.0
"#4: def tknz + lowercase & our own embds (win 5, min_cnt 5)",0.261,0.261,0.314556,0.361,0.331864,0.414,0.369950,0.604,0.400832,0.844,0.417272,1.0
#5: word_tokenize from nltk + lowercase & stackoverflow embds,0.399,0.399,0.486750,0.566,0.508322,0.633,0.554418,0.858,0.569235,0.969,0.572556,1.0
"#6: word_tokenize from nltk + lowercase & our own embds (win 5, min_cnt 5)",0.229,0.229,0.282194,0.330,0.297580,0.378,0.335722,0.571,0.369668,0.838,0.386788,1.0
#7: improved tokenizer + lowercase & stackoverflow embds,0.411,0.411,0.494767,0.573,0.515573,0.638,0.561398,0.864,0.575714,0.971,0.578823,1.0
"#8: improved tokenizer + lowercase & our own embds (win 5, min_cnt 5)",0.250,0.250,0.304011,0.351,0.320441,0.402,0.357935,0.589,0.389876,0.838,0.406951,1.0
#9: word_tokenize from nltk + lowercase + punct filter + split by dot & stackoverflow embds,0.408,0.408,0.493286,0.573,0.513590,0.636,0.559540,0.862,0.573987,0.970,0.577207,1.0


### Complete table of experiments (sorted by DCG@1 descending)

In [126]:
df.sort_values(by='DCG@1', ascending=False)

,DCG@1,Hits@1,DCG@5,Hits@5,DCG@10,Hits@10,DCG@100,Hits@100,DCG@500,Hits@500,DCG@1000,Hits@1000
experiment,,,,,,,,,,,,
#11: imprvd tok v2 (wo forward slash & hyphen) + lowercase & & stackoverflow embds,0.418,0.418,0.504088,0.584,0.525277,0.650,0.571488,0.877,0.584305,0.974,0.587106,1.0
#13: imprvd tok v2 + lowercase + rmv stp wrds & stackoverflow embds,0.418,0.418,0.503815,0.583,0.525351,0.650,0.571787,0.878,0.584471,0.974,0.587272,1.0
#3: def tknz + lowercase & stackoverflow embds,0.415,0.415,0.502495,0.582,0.524731,0.651,0.570256,0.874,0.583425,0.973,0.586335,1.0
#7: improved tokenizer + lowercase & stackoverflow embds,0.411,0.411,0.494767,0.573,0.515573,0.638,0.561398,0.864,0.575714,0.971,0.578823,1.0
#17: imprvd tok v2 + lowercase + rmv stp wrds + WordNetLemmatizer & stackoverflow embds,0.409,0.409,0.495106,0.573,0.515342,0.636,0.563622,0.872,0.576218,0.967,0.579763,1.0
#9: word_tokenize from nltk + lowercase + punct filter + split by dot & stackoverflow embds,0.408,0.408,0.493286,0.573,0.513590,0.636,0.559540,0.862,0.573987,0.970,0.577207,1.0
"#22: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & our own embds (win 5, min_cnt 2)",0.400,0.400,0.483302,0.554,0.500575,0.607,0.535632,0.779,0.548788,0.882,0.561235,1.0
#5: word_tokenize from nltk + lowercase & stackoverflow embds,0.399,0.399,0.486750,0.566,0.508322,0.633,0.554418,0.858,0.569235,0.969,0.572556,1.0
"#16: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & our own embds (win 5, min_cnt 5)",0.397,0.397,0.478834,0.547,0.496430,0.601,0.532493,0.777,0.545635,0.880,0.558281,1.0


### Experiments with only embeddings pretrained on stackoverflow (sorted by DCG@1 descending)

In [127]:
metrics_with_soe = {}
metrics_with_our_embds = {}

for label, values in metrics.items():
    metrics_with_soe[label] = values_with_soe = []
    metrics_with_our_embds[label]  = values_with_our_embds = []
    for exp_desc, value in zip(metrics['experiment'], values):
        if "stackoverflow" in exp_desc:
            values_with_soe.append(value)
        else:
            values_with_our_embds.append(value)

In [128]:
soe_df = pd.DataFrame(metrics_with_soe)
soe_df.set_index('experiment', inplace=True)
soe_df.sort_values(by='DCG@1', ascending=False)

,DCG@1,Hits@1,DCG@5,Hits@5,DCG@10,Hits@10,DCG@100,Hits@100,DCG@500,Hits@500,DCG@1000,Hits@1000
experiment,,,,,,,,,,,,
#11: imprvd tok v2 (wo forward slash & hyphen) + lowercase & & stackoverflow embds,0.418,0.418,0.504088,0.584,0.525277,0.650,0.571488,0.877,0.584305,0.974,0.587106,1.0
#13: imprvd tok v2 + lowercase + rmv stp wrds & stackoverflow embds,0.418,0.418,0.503815,0.583,0.525351,0.650,0.571787,0.878,0.584471,0.974,0.587272,1.0
#3: def tknz + lowercase & stackoverflow embds,0.415,0.415,0.502495,0.582,0.524731,0.651,0.570256,0.874,0.583425,0.973,0.586335,1.0
#7: improved tokenizer + lowercase & stackoverflow embds,0.411,0.411,0.494767,0.573,0.515573,0.638,0.561398,0.864,0.575714,0.971,0.578823,1.0
#17: imprvd tok v2 + lowercase + rmv stp wrds + WordNetLemmatizer & stackoverflow embds,0.409,0.409,0.495106,0.573,0.515342,0.636,0.563622,0.872,0.576218,0.967,0.579763,1.0
#9: word_tokenize from nltk + lowercase + punct filter + split by dot & stackoverflow embds,0.408,0.408,0.493286,0.573,0.513590,0.636,0.559540,0.862,0.573987,0.970,0.577207,1.0
#5: word_tokenize from nltk + lowercase & stackoverflow embds,0.399,0.399,0.486750,0.566,0.508322,0.633,0.554418,0.858,0.569235,0.969,0.572556,1.0
#15: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & stackoverflow embds,0.323,0.323,0.401690,0.474,0.423024,0.541,0.474851,0.797,0.494546,0.948,0.500085,1.0
#1: def tknz & stackoverflow embds,0.285,0.285,0.341622,0.393,0.359693,0.449,0.405681,0.679,0.431275,0.879,0.443923,1.0


### Experiments with only our own embeddings (sorted by DCG@1 descending)

In [129]:
our_embds_df = pd.DataFrame(metrics_with_our_embds)
our_embds_df.set_index('experiment', inplace=True)
our_embds_df.sort_values(by='DCG@1', ascending=False)

,DCG@1,Hits@1,DCG@5,Hits@5,DCG@10,Hits@10,DCG@100,Hits@100,DCG@500,Hits@500,DCG@1000,Hits@1000
experiment,,,,,,,,,,,,
"#22: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & our own embds (win 5, min_cnt 2)",0.400,0.400,0.483302,0.554,0.500575,0.607,0.535632,0.779,0.548788,0.882,0.561235,1.0
"#16: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & our own embds (win 5, min_cnt 5)",0.397,0.397,0.478834,0.547,0.496430,0.601,0.532493,0.777,0.545635,0.880,0.558281,1.0
"#19: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & our own embds (win 2, min_cnt 5)",0.397,0.397,0.478834,0.547,0.496430,0.601,0.532493,0.777,0.545635,0.880,0.558281,1.0
"#20: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & our own embds (win 10, min_cnt 5)",0.397,0.397,0.478834,0.547,0.496430,0.601,0.532493,0.777,0.545635,0.880,0.558281,1.0
"#21: imprvd tok v2 + lowercase + rmv stp wrds + PorterStemmer & our own embds (win 5, min_cnt 10)",0.392,0.392,0.472841,0.540,0.491720,0.598,0.527650,0.773,0.541417,0.881,0.553944,1.0
"#18: imprvd tok v2 + lowercase + rmv stp wrds + WordNetLemmatizer & our own embds (win 5, min_cnt 5)",0.390,0.390,0.455842,0.517,0.473777,0.572,0.509851,0.750,0.523453,0.856,0.538567,1.0
"#14: imprvd tok v2 + lowercase + rmv stp wrds & our own embds (win 5, min_cnt 5)",0.367,0.367,0.437772,0.501,0.455323,0.555,0.487382,0.714,0.504018,0.845,0.520267,1.0
"#4: def tknz + lowercase & our own embds (win 5, min_cnt 5)",0.261,0.261,0.314556,0.361,0.331864,0.414,0.369950,0.604,0.400832,0.844,0.417272,1.0
"#12: imprvd tok v2 (wo forward slash & hyphen) + lowercase & our own embds (win 5, min_cnt 5)",0.261,0.261,0.312750,0.358,0.330406,0.412,0.368861,0.604,0.399832,0.845,0.416154,1.0


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:
Q: Какой принцип токенизации даёт качество лучше и почему?

A: Лучше работает самописный метод, который включает в токены часть символов пунктуации. Такова специфика домена: много терминов, ключевых слов и т.п. включают подобные символы, что немного отличается от обычной лексики.

Q: Помогает ли нормализация слов?

A: Помогает, если при обучении эмбеддингов её тоже использовали.

Q: Какие эмбеддинги лучше справляются с задачей и почему?

A: Предобученные. Причина, видимо, в том, что домен тот же, но сам корпус значительно больше.

Q: Почему получилось плохое качество решения задачи?

A: В принципе плохое качество или плохое качество наших эмбеддингов по сравнению с предобученными? Если второе, то опять же, вероятно, из-за разницы в объеме тренировочных данных, хотя максимальный результат наших (#16 или #22) относительно близко подобрался к максимальному результату на предобученных (#11 или #13). Если первое, то, наверное, причина в качестве тренировочных и/или валидационных данных или же в самом методе: простого среднего векторов слов недостаточно.

Q: Предложите свой подход к решению задачи.

A:
- Взглянуть на данные
- Подобрать удачную токенизацию
- Отсеять пунктуацию, стоп-слова
- Применить ту или иную нормализацию
- Обучить эмбеддинги
- Если используются предобученные эмбеддинги, то важно предыдущие пункты согласовать с ними
- Провести валидацию
- Если результат не устраивает, то
    - что-либо поменять в предыдущих пунктах (изменить токенизацию, изменить/убрать нормализацию)
    - потюнить параметры обучения эмбеддингов
    - найти тренировочные данные получше